In [95]:
import argparse
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import pickle
import platform
import os

In [96]:
# from google.colab import drive
# drive.mount('/content/drive')

In [97]:
# !git clone https://github.com/vita-epfl/DLAV-2024.git
path = os.getcwd() + '/test_batch'

In [98]:
# Write the location of the saved weight relative to this notebook. Assume that they are in the same directory
### Path to Model Weights 
softmax_weights = os.getcwd() + '/softmax_weights.pkl'
pytorch_weights = os.getcwd() + '/linearClassifier_pytorch.ckpt'

**TODO:** Copy your code from the Softmax Notebook to their corresponding function

In [99]:

def softmax_loss_vectorized(W, X, y):
    """
  Softmax loss function, vectorized version.
  Inputs and outputs are the same as softmax_loss_naive.
  """
    # Initialize the loss and gradient to zero.
    loss = 0.0
    dW = np.zeros_like(W)

    #############################################################################
    # TODO: Compute the softmax loss and its gradient using no explicit loops.  #
    # Store the loss in loss and the gradient in dW. If you are not careful     #
    # here, it is easy to run into numeric instability. Don't forget the        #
    # regularization!                                                           #
    #############################################################################
    num_samples = y.shape[0]
    # calculate shifted score (max = 0) to avoid exp overflow
    scores = X.dot(W) - np.max(X.dot(W), axis=1, keepdims=True)
    exp_scores = np.exp(scores)
    softmax = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)

    # Compute cross-entropy of softmax probabilities
    correct_logprobs = -np.log(softmax[np.arange(num_samples), y])
    loss = np.sum(correct_logprobs) / num_samples

    # Compute the gradient
    softmax[np.arange(num_samples), y] -= 1
    softmax /= num_samples
    dW = X.T.dot(softmax)
    #############################################################################
    #                          END OF YOUR CODE                                 #
    #############################################################################
    
    return loss, dW

class LinearClassifier(object):

    def __init__(self):
        self.W = None


    def train(self, X, y, learning_rate=1e-3, num_iters=30000,
                batch_size=200, verbose=False):
        """
        Train this linear classifier using stochastic gradient descent.

        Inputs:
        - X: A numpy array of shape (N, D) containing training data; there are N
          training samples each of dimension D.
        - y: A numpy array of shape (N,) containing training labels; y[i] = c
          means that X[i] has label 0 <= c < C for C classes.
        - learning_rate: (float) learning rate for optimization.
        - num_iters: (integer) number of steps to take when optimizing
        - batch_size: (integer) number of training examples to use at each step.
        - verbose: (boolean) If true, print progress during optimization.

        Outputs:
        A list containing the value of the loss function at each training iteration.
        """
        
        num_train, dim = X.shape
        num_classes = np.max(y) + 1 # assume y takes values 0...K-1 where K is number of classes
        
        if self.W is None:
            # lazily initialize W
            self.W = 0.001 * np.random.randn(dim, num_classes)

        # Run stochastic gradient descent to optimize W
        loss_history = []
        for it in range(num_iters):
            X_batch = None
            y_batch = None

            #########################################################################
            # TODO:                                                                 #
            # Sample batch_size elements from the training data and their           #
            # corresponding labels to use in this round of gradient descent.        #
            # Store the data in X_batch and their corresponding labels in           #
            # y_batch; after sampling X_batch should have shape (dim, batch_size)   #
            # and y_batch should have shape (batch_size,)                           #
            #                                                                       #
            # Hint: Use np.random.choice to generate indices. Sampling with         #
            # replacement is faster than sampling without replacement.              #
            #########################################################################
            idx = np.random.choice(X.shape[0], batch_size, replace=True)
            X_batch = X[idx]
            y_batch = y[idx]
            #########################################################################
            #                       END OF YOUR CODE                                #
            #########################################################################

            # evaluate loss and gradient
            loss, grad = self.loss(X_batch, y_batch, reg)
            loss_history.append(loss)

            # perform parameter update
            #########################################################################
            # TODO:                                                                 #
            # Update the weights using the gradient and the learning rate.          #
            #########################################################################
            self.W = self.W - learning_rate * grad
            #########################################################################
            #                       END OF YOUR CODE                                #
            #########################################################################

            if verbose and it % 100 == 0:
                print('iteration %d / %d: loss %f' % (it, num_iters, loss))


        return loss_history
    


    def predict(self, X):
        """
        Use the trained weights of this linear classifier to predict labels for
        data points.

        Inputs:
        - X: A numpy array of shape (N, D) containing training data; there are N
          training samples each of dimension D.

        Returns:
        - y_pred: Predicted labels for the data in X. y_pred is a 1-dimensional
          array of length N, and each element is an integer giving the predicted
          class.
        """

        ###########################################################################
        # TODO:                                                                   #
        # Implement this method. Store the predicted labels in y_pred.            #
        ###########################################################################
        y_pred = np.argmax(X.dot(self.W), axis=1)
        ###########################################################################
        #                           END OF YOUR CODE                              #
        ###########################################################################
        return y_pred

    def loss(self, X_batch, y_batch):
        """
        Compute the loss function and its derivative. 
        Subclasses will override this.

        Inputs:
        - X_batch: A numpy array of shape (N, D) containing a minibatch of N
          data points; each point has dimension D.
        - y_batch: A numpy array of shape (N,) containing labels for the minibatch.


        Returns: A tuple containing:
        - loss as a single float
        - gradient with respect to self.W; an array of the same shape as W
        
         e = y_batch - np.dot(X_batch, self.W) 
        
        loss = np.dot(e.T, e)
        grad = -np.dot(x_batch.T,e) / x_batch.shape[0]
  
        return loss, grad

        """

        pass
        


class Softmax(LinearClassifier):
    """ A subclass that uses the Softmax + Cross-entropy loss function """

    def loss(self, X_batch, y_batch):
        return softmax_loss_vectorized(self.W, X_batch, y_batch)

**TODO:** Copy the model you created from the Pytorch Notebook

In [100]:
class Net(torch.nn.Module):
    def __init__(self, n_feature, n_hidden, n_output, dropout=True):
        super(Net, self).__init__()
        
        ################################################################################
        # TODO:                                                                        #
        # Define 2 or more different layers of the neural network                      #
        ################################################################################
        self.fc1 = nn.Linear(n_feature, n_hidden)
        self.fc2 = nn.Linear(n_hidden, n_hidden)
        self.fc3 = nn.Linear(n_hidden, n_output)

        if dropout:
            self.dropout = nn.Dropout(p=0.25)
        ################################################################################
        #                              END OF YOUR CODE                                #
        ################################################################################


    def forward(self, x):
        x = x.view(x.size(0),-1)
        ################################################################################
        # TODO:                                                                        #
        # Set up the forward pass that the input data will go through.                 #
        # A good activation function betweent the layers is a ReLu function.           #
        ################################################################################
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.fc3(x)
        x = F.softmax(x, dim=1)
        ################################################################################
        #                              END OF YOUR CODE                                #
        ################################################################################
        return x

**TODO**: Follow the instructions in each of the following methods. **Note that these methods should return a 1-D array of size N where N is the number of data samples. The values should be the predicted classes [0,...,9].**



In [101]:
def predict_usingPytorch(X):
    #########################################################################
    # TODO:                                                                 #
    # - Create your model                                                   #
    # - Load your saved model                                               #
    # - Do the operation required to get the predictions                    #
    # - Return predictions in a numpy array (hint: return "argmax")         #
    #########################################################################
    net = Net(n_feature=32*32*3, n_hidden=1000, n_output=10)
    checkpoint = torch.load(pytorch_weights)
    net.load_state_dict(checkpoint)

    output = net(X)

    y_pred = torch.argmax(output, dim=1)
    #########################################################################
    #                       END OF YOUR CODE                                #
    #########################################################################
    return y_pred.numpy()

def predict_usingSoftmax(X):
    #########################################################################
    # TODO:                                                                 #
    # - Load your saved model into the weights of Softmax                   #
    # - Do the operation required to get the predictions                    #
    # - Return predictions in a numpy array                                 #
    #########################################################################
    with open(softmax_weights, 'rb') as f:
        W = pickle.load(f)
    sm = Softmax()
    sm.W = W.copy()

    y_pred = sm.predict(X)
    #########################################################################
    #                       END OF YOUR CODE                                #
    #########################################################################
    return y_pred

This method loads the test dataset to evaluate the model.

In [102]:
## Read DATA
def load_pickle(f):
    version = platform.python_version_tuple()
    if version[0] == '2':
        return  pickle.load(f)
    elif version[0] == '3':
        return  pickle.load(f, encoding='latin1')
    raise ValueError("invalid python version: {}".format(version))

def load_CIFAR_batch(filename):
  """ load single batch of cifar """
  with open(filename, 'rb') as f:
    datadict = load_pickle(f)
    X = datadict['data']
    Y = datadict['labels']
    X = X.reshape(10000, 3, 32, 32).transpose(0,2,3,1).astype("float")
    Y = np.array(Y)
    return X, Y
test_filename = path
X,Y = load_CIFAR_batch(test_filename)

This code snippet prepares the data for the different models. If you modify data manipulation in your notebooks, make sure to include them here. 

In [103]:
## Data Manipulation

mean = np.array([0.4914, 0.4822, 0.4465])
std = np.array([0.2023, 0.1994, 0.2010])
X = np.divide(np.subtract( X/255 , mean[np.newaxis,np.newaxis,:]), std[np.newaxis,np.newaxis,:])

X_pytorch = torch.Tensor(np.moveaxis(X,-1,1))
X_softmax = np.reshape(X, (X.shape[0], -1))
X_softmax = np.hstack([X_softmax, np.ones((X_softmax.shape[0], 1))])


Runs evaluation on the Pytorch and softmax model. **Be careful that *prediction_pytorch* and *prediction_softmax* are 1-D array of size N where N is the number of data samples. The values should be the predicted class [0,...,9]**

---



In [104]:
## Run Prediction
prediction_pytorch = predict_usingPytorch(X_pytorch)
prediction_softmax = predict_usingSoftmax(X_softmax)

## Run Evaluation
acc_softmax = sum(prediction_softmax == Y)/len(X)
acc_pytorch = sum(prediction_pytorch == Y)/len(X)
print("Softmax= %f ... Pytorch= %f"%(acc_softmax, acc_pytorch))

Softmax= 0.380400 ... Pytorch= 0.496600
